In [1]:
GAME_ID = "0022300869"

event_df = EventProcessor.load_games()
tracking_df = TrackingProcessor.load_games()
possession_df = PossessionProcessor.load_games()

In [2]:
shot_rebound_df = ActionProcessor.extract_shots_and_rebounds(event_df, tracking_df)
shot_rebound_df
shot_rebound_classified_df = FeatureUtil.classify_shot_locations(shot_rebound_df, possession_df, FeatureUtil.classify_shot_region)
shot_rebound_classified_df.to_csv('./../data/results/shot_rebound_classified.csv', index=False)

In [3]:
hexbin_region_data_df = StatsUtil.generate_region_hexbin_data(shot_rebound_classified_df, ShotRegionUtil.regions)
hexbin_region_data_df.to_csv('./../data/results/rebound_hexmap_distributions.csv')

In [4]:
#reb_chances_df = StatsUtil.assign_rebound_chances_to_shots(shot_rebound_classified_df, tracking_df, hexbin_region_data_df)
#reb_chances_df.to_csv('./../data/results/reb_chances.csv')

100%|██████████| 810/810 [44:42<00:00,  3.31s/it]  


In [5]:
off_rebounds_df = EventProcessor.extract_off_rebounds(event_df)
oreb_ppp = FeatureUtil.calculate_oreb_ppp(event_df, off_rebounds_df)
oreb_ppp

1.1025641025641026

In [8]:
true_points_df = pd.read_csv('./../data/results/true_points.csv', dtype={'made': pd.BooleanDtype()})
reb_chances_df = pd.read_csv('./../data/results/reb_chances.csv', dtype={'dReb': pd.BooleanDtype()})

true_impact_points_df = StatsUtil.assign_oreb_expected_points_to_shots(true_points_df, reb_chances_df, oreb_ppp=1.1025641025641026)
true_impact_points_df.to_csv('./../data/results/true_impact_points.csv', index=False)

,points_produced,true_points_produced,true_impact_points_produced
count,1958.000000,1958.000000,1958.000000
mean,1.044433,1.173136,1.459532
std,1.187814,1.188496,1.215403
min,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.197792
50%,0.000000,1.000000,1.709318
75%,2.000000,2.000000,2.412185
max,3.000000,4.000000,4.749744


In [ ]:
true_impact_points_df = pd.read_csv('./../data/results/true_impact_points.csv')
shot_classification_df = pd.read_csv('./../data/results/shot_rebound_classified.csv', dtype={'made': pd.BooleanDtype()})

true_impact_points_df[['points_produced', 'true_points_produced', 'true_impact_points_produced']].describe()
shot_classification_df = shot_classification_df.rename(columns={"shot_time": "wcTime"})
merged_df = true_impact_points_df.merge(
    shot_classification_df[['gameId', 'playerId', 'wcTime', 'shot_classification']],
    on=['gameId', 'playerId', 'wcTime'],
    how='left'
)
merged_df.to_csv('./../data/results/true_impact_points_classified.csv', index=False)

In [ ]:
reb_chances_df = pd.read_csv('./../data/results/reb_chances.csv', dtype={'dReb': pd.BooleanDtype()})

FeatureUtil.calculate_shot_statistics_by_region(merged_df)

In [ ]:
misses = reb_chances_df.loc[reb_chances_df['made'] == False]
FeatureUtil.calculate_brier_score_loss(misses)

In [ ]:
VisUtil.plot_auc(misses['dReb'], misses['def_reb_chance'], "Off/Def Rebounding Predictions")